In [1]:
import numpy as np
import tensorflow as tf

# https://github.com/patrick-kidger/tools
import tools

from base import fenics as fc
from base import ml

In [2]:
# Set to False to use existing weights for the neural network.
# Set to True to continue training the weights from where we left off last time.
TRAIN = True

In [ ]:
# DNN hyperparameters
hidden_units = [100] * 7
# i.e. the number of fine grid points:
logits = (ml.fine_grid_fineness.t + 1) * (ml.fine_grid_fineness.x + 1)
processor = ml.ScaleCenter()
activation = tf.nn.crelu

### Training hyperparameters #1
drop_rate = 0.0
drop_type = 'dropout'
gradient_clip = 1.0  # May be set to None to turn off gradient clipping

### Logging/saving hyperparameters
log_steps = 1000  # How many steps to print the current loss.
model_dir_base = './saved_models/grid_general/'
uuid = None
model_dir = ml.model_dir_str(model_dir_base, hidden_units, logits,
                             processor=processor, activation=activation, 
                             uuid=uuid)

dnn_factory = ml.DNNFactory(hidden_units, logits, drop_rate=drop_rate,
                            drop_type=drop_type, model_dir=model_dir, 
                            log_steps=log_steps, activation=activation, 
                            gradient_clip=gradient_clip, batch_norm=False,
                            processor=processor)

max_steps = 0
if TRAIN:
    # Start off without using FEniCS solutions because they're so much
    # slower to generate
    gen_one_data = fc.GenGeneralSolutionOnGrid(fenics_num=0)
    
    ### Training hyperparameters #2
    batch_size = 50
    max_steps += 5000
    with ml.BatchData.context(gen_one_data=gen_one_data, batch_size=batch_size) as train_input_fn:
        dnn = dnn_factory()
        dnn.train(input_fn=train_input_fn, max_steps=max_steps)

    batch_size = 500
    max_steps += 5000
    with ml.BatchData.context(gen_one_data=gen_one_data, batch_size=batch_size) as train_input_fn:
        dnn = dnn_factory()
        dnn.train(input_fn=train_input_fn, max_steps=max_steps)
    
    # Start using FEniCS solutions
    gen_one_data = fc.GenGeneralSolutionOnGrid()
    
    batch_size = 500
    max_steps += 3000
    with ml.BatchData.context(gen_one_data=gen_one_data, batch_size=batch_size) as train_input_fn:
        dnn = dnn_factory()
        dnn.train(input_fn=train_input_fn, max_steps=max_steps)
        
    batch_size = 1500
    max_steps += 1000
    with ml.BatchData.context(gen_one_data=gen_one_data, batch_size=batch_size) as train_input_fn:
        dnn = dnn_factory()
        dnn.train(input_fn=train_input_fn, max_steps=max_steps)

INFO:tensorflow:Using config: {'_model_dir': './saved_models/grid_general/100x7_121__ScaleCenter_crelu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 1000, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f642c66aac8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./saved_models/grid_general/100x7_121__ScaleCenter_crelu/model.ckpt-2133
INFO:tensorflow:Running local_init_op.
INFO:tens